**Table of contents**<a id='toc0_'></a>    
- [1. About the Notebook](#toc1_)    
- [2. Libraries and Packages](#toc2_)    
- [3. Connecting In Snowflake](#toc3_)    
- [4. Defining Functions](#toc4_)    
  - [4.1. Text Cleaning](#toc4_1_)    
  - [4.2. Tokenizing](#toc4_2_)    
  - [4.3. Remove Stopwords](#toc4_3_)    
  - [4.4. Count Words in each product and discard irrelevant products](#toc4_4_)    
  - [4.5. Count Vectorizer](#toc4_5_)    
  - [4.6. One Hot Encoding](#toc4_6_)    
  - [4.7. Recommendations with WALS](#toc4_7_)    
- [5. Loading Data and Applying all Functions](#toc5_)    
  - [5.1. Extracting list of all Main Categories](#toc5_1_)    
  - [5.2. Producing a Recommendation dataframe to each Main Category](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[1. About the Notebook](#toc0_)

Notebook that creates personalized recommendations

# <a id='toc2_'></a>[2. Libraries and Packages](#toc0_)

In [1]:
import snowflake.connector
import glob, os

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

from time import sleep

import nltk
nltk.download("punkt")
import spacy

from loguru import logger


import spacy
from spacy.lang.en.examples import sentences 
from tqdm import tqdm
import itertools

/home/brunnokalyxton/anaconda3/lib/python3.9/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (8.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
[nltk_data] Downloading package punkt to
[nltk_data]     /home/brunnokalyxton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# <a id='toc3_'></a>[3. Connecting In Snowflake](#toc0_)

In [2]:
conn = snowflake.connector.connect(
    user='***********',
    password='***************',
    account='***************',
    warehouse='ANALYTICS_WH',
    database='AMAZON',
    schema='AMZ_DATA_GOLD',
    role = 'ANALYSTS'
)

# <a id='toc4_'></a>[4. Defining Functions](#toc0_)

In [14]:
logger.info('Defining chain_recommendation function...')

def chain_recommendation(df):
    product_id = df['product_id'].values.tolist()

    all_recommendations = df['recommendations'].values.tolist()
    
    chained_recommendations = list(itertools.chain.from_iterable(zip(*all_recommendations)))        
        
    
    return pd.Series({'product_id': product_id, 'recommendations': chained_recommendations})

2023-08-05 01:41:05.714 | INFO     | __main__:<module>:1 - Defining chain_recommendation function...


# <a id='toc5_'></a>[5. Loading Category Data](#toc0_)

In [15]:
query1 = """
WITH CTE AS (
    SELECT 
        REV.REVIEWER_ID,
        REV.ASIN,
        REV.REVIEW_TEXT,
        PROD.MAIN_CATEGORY,
        AVG(REV.OVERALL) OVER (PARTITION BY REV.ASIN) AS "PRODUCT_AVG_RATING" 
    FROM
        PRODUCTS_REVIEWS AS REV
    INNER JOIN 
        PRODUCTS AS PROD ON REV.ASIN = PROD.ASIN
    WHERE 
        REV.REVIEW_TEXT IS NOT NULL AND 
        REV.REVIEW_TEXT <> '' 
)
SELECT 
    DISTINCT MAIN_CATEGORY
FROM 
    CTE
WHERE 
    PRODUCT_AVG_RATING >=4 AND
    MAIN_CATEGORY IS NOT NULL AND 
    MAIN_CATEGORY <> ''
GROUP BY
    1
"""

In [16]:
category = pd.read_sql_query(query1, conn)

/tmp/ipykernel_7010/3386730848.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  category = pd.read_sql_query(query1, conn)


In [17]:
category_list = category.MAIN_CATEGORY.values.tolist()

# <a id='toc6_'></a>[6. Personalized Recommendations](#toc0_)

In [19]:
for i in category_list:
    try:
        df_name = f'recommendations_{i}'  # Creating a name for the DataFrame
        globals()[df_name]  = pd.read_parquet(
            "../output/similarity/recommendations_{i}.parquet".format(i=i)
        )
    
        query = """
        WITH CTE AS (
            SELECT 
                REV.REVIEWER_ID,
                REV.ASIN,
                REV.REVIEW_TEXT,
                PROD.MAIN_CATEGORY,
                AVG(REV.OVERALL) OVER (PARTITION BY REV.ASIN) AS "PRODUCT_AVG_RATING" 
            FROM
                PRODUCTS_REVIEWS AS REV
            INNER JOIN 
                PRODUCTS AS PROD ON REV.ASIN = PROD.ASIN
            WHERE 
                REV.REVIEW_TEXT IS NOT NULL AND 
                REV.REVIEW_TEXT <> '' AND
                PROD.MAIN_CATEGORY = '{i}'
        )
        SELECT 
            CTE.REVIEWER_ID,
            LISTAGG(DISTINCT CTE.ASIN, ',') AS PRODUCT_BOUGHT
        FROM 
            CTE
        WHERE 
            CTE.PRODUCT_AVG_RATING >=4
        GROUP BY
            1
        """.format(i=i)
        
        purchases = pd.read_sql_query(query, conn)

        purchases = purchases.rename({'REVIEWER_ID': 'REVIEWER_ID', 'PRODUCT_BOUGHT': 'product_id'}, axis=1)
        purchases['product_id'] = purchases['product_id'].values.tolist()
        multiple_purchases_casted = purchases.copy()
        multiple_purchases_casted['product_id'] = multiple_purchases_casted['product_id'].apply(lambda x: x.split(','))
        multiple_purchases_exploded = multiple_purchases_casted.explode('product_id')
        purchases_recommendations_wide = pd.merge(multiple_purchases_exploded, globals()[df_name], how='inner', on = 'product_id')
        
        if len(purchases_recommendations_wide) > 0:

            tqdm.pandas(desc="Processing personalized recommendations for {i}...".format(i=i))
            personalized = purchases_recommendations_wide.groupby('REVIEWER_ID').progress_apply(chain_recommendation).reset_index()

            personalized.to_parquet(
                    path="../output/personalized/personalized_{i}.parquet".format(i=i),
                    engine="auto"
                )  
        else:
            pass
    except FileNotFoundError:
    
        print(f"File 'recommendations_{i}' not found.".format(i=i))
    
        continue

/tmp/ipykernel_7010/3470674740.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  purchases = pd.read_sql_query(query, conn)
Processing personalized recommendations for Movies & Tv...: 100%|██████████| 1563/1563 [00:00<00:00, 4332.30it/s]
/tmp/ipykernel_7010/3470674740.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  purchases = pd.read_sql_query(query, conn)


File 'recommendations_Alexa Skills' not found.
File 'recommendations_Audible Audiobooks' not found.
File 'recommendations_Portable Audio & Accessories' not found.
File 'recommendations_Buy A Kindle' not found.
File 'recommendations_Software' not found.


Processing personalized recommendations for Amazon Fashion...: 100%|██████████| 862/862 [00:00<00:00, 4118.44it/s]
/tmp/ipykernel_7010/3470674740.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  purchases = pd.read_sql_query(query, conn)


File 'recommendations_Collectible Coins' not found.
File 'recommendations_Computers' not found.


Processing personalized recommendations for Cell Phones & Accessories...: 100%|██████████| 1041/1041 [00:00<00:00, 3931.12it/s]
/tmp/ipykernel_7010/3470674740.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  purchases = pd.read_sql_query(query, conn)


...

File 'recommendations_Handmade' not found.


Processing personalized recommendations for Sports Collectibles...: 100%|██████████| 169/169 [00:00<00:00, 3225.86it/s]
/tmp/ipykernel_7010/3470674740.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  purchases = pd.read_sql_query(query, conn)


File 'recommendations_Amazon Fire Tv' not found.
File 'recommendations_Home & Business Services' not found.
File 'recommendations_Magazine Subscriptions' not found.


Processing personalized recommendations for Industrial & Scientific...: 100%|██████████| 32097/32097 [00:16<00:00, 1907.47it/s]

File 'recommendations_Car Electronics' not found.
File 'recommendations_Entertainment' not found.


# <a id='toc7_'></a>[7. Merging Recommendations Dataframes](#toc0_)

In [3]:
# Returns a dataframe that contains all of the directory's parquet files
def combine_directory_of_parquet(directory='../output/similarity/**.parquet', recursive=True, columns=[]):

    # Create an empty dataframe to hold our combined data
    merged_df = pd.DataFrame(columns=columns)

    # Iterate over all of the files in the provided directory and
    # configure if we want to recursively search the directory
    for filename in glob.iglob(pathname=directory, recursive=recursive):

        # Check if the file is actually a file (not a directory) and make sure it is a parquet file
        if os.path.isfile(filename):
            try:
                # Perform a read on our dataframe
                temp_df = pd.read_parquet(filename)

                # Attempt to merge it into our combined dataframe
                merged_df = pd.concat([merged_df, temp_df], ignore_index=True)
                # merged_df = merged_df.append(temp_df, ignore_index=True)

            except Exception as e:
                print('Skipping {} due to error: {}'.format(filename, e))
                continue
        else:
            print('Not a file {}'.format(filename))

    # Return the result!
    return merged_df

In [4]:
# Columns for similarity recommendations dataframe
columns = ['product_id', 'recommendations', 'MAIN_CATEGORY']


similarity_recommendations = combine_directory_of_parquet(directory='../output/similarity/**.parquet', recursive=True, columns=columns)

# Saving all similarity recommendations
similarity_recommendations.to_parquet(
                    path="../output/similarity/similarity_recommendations.parquet",
                    engine="auto"
                ) 

In [5]:
# Columns for personalized recommendations dataframe
columns = ['REVIEWER_ID', 'product_id', 'recommendations']

# You can modify the directory path below, the asterisks are wildcard selectors to match any file.
personalized_recommendations = combine_directory_of_parquet(directory='../output/personalized/**.parquet', recursive=True, columns=columns)

# Saving all personalized recommendations
personalized_recommendations.to_parquet(
                    path="../output/personalized/personalized_recommendations.parquet",
                    engine="auto"
                ) 

# <a id='toc8_'></a>[8. Sending Data To Snowflake](#toc0_)

In [ ]:
# personalized_recommendations
# similarity_recommendations